# 经销商会话时长分析

1、回复状态：如果1个会话ID的最后1条发送方为"用户"，则返回"未回复"；如果1个会话ID的最后1条发送方为"商家"，则返回"已回复"

2、对话持续时间(s)：每个会话ID的最后1条消息发送时间减去第1条消息的发送时间

3、首次应答时间(s)：每个会话ID的第1次"商家"发送时间减去第1次"用户"发送时间

4、对话开始时间(s)：每个会话ID的第1次消息发送时间

5、平均回复时长(s)

In [1]:
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
# 读取数据
df = pd.read_excel("./file/message.xlsx")
df.消息发送时间 = pd.to_datetime(df['消息发送时间'])
result = []
for (agecny, sess_id), data in df.groupby(["经销商", "会话ID"]):
#     if agecny == '佩戴' and sess_id == 145:
        # 回复状态
        status = "已回复" if data.iloc[-1].发送方 == "商家" else "未回复"
        # 对话持续时间
        stay_time = (data.消息发送时间.max() - data.消息发送时间.min()).total_seconds()
        # 首次应答时间
        t = data.query('发送方 == "商家"')
        first_res_time = (t.iloc[0].消息发送时间 - data.query('发送方 == "用户"').iloc[0].消息发送时间).total_seconds() if t.shape[0] > 0 else None                                                                                                           
        # 对话开始时间
        start_time = data.query('发送方 == "用户"').iloc[0].消息发送时间
        # 平均回复时长
            ## 找到用户发送的消息的id,用户连续发送则取第一个id
        user_id_list = list(data.query('发送方 == "用户"').index)
        del_list = [i for i in range(1, len(user_id_list)) if user_id_list[i] - user_id_list[i-1] == 1]
        user_id_list = [n for i,n in enumerate(user_id_list) if i not in del_list]
            ## 找到商家发送消息的id
        seller_id_list = list(data.query('发送方 == "商家"').index)
        compare_start_end = [(i, min(filter(lambda x:x>i, seller_id_list))) for i in user_id_list if list(filter(lambda x:x>i, seller_id_list))]                 
        time_list = [(data.loc[i[1]].消息发送时间 - data.loc[i[0]].消息发送时间).total_seconds() for i in compare_start_end]
        ave_res_time = reduce(lambda x,y:x+y,time_list)/len(compare_start_end) if time_list else None 
        result.append([agecny, sess_id, status, stay_time, first_res_time, start_time, ave_res_time]) 
res = pd.DataFrame(result, columns=["经销商", "会话ID", "回复状态","对话持续时间", "首次应答时间", "对话开始时间", "平均回复时长"])
res = res.round(2).sort_values("会话ID", ignore_index=True)
res

,经销商,会话ID,回复状态,对话持续时间,首次应答时间,对话开始时间,平均回复时长
0,佩戴,1,已回复,105.0,5.0,2022-08-05 14:33:48,10.0
1,佩戴,2,已回复,43.0,6.0,2022-08-04 10:42:36,13.5
2,佩戴,3,已回复,144.0,63.0,2022-08-01 10:16:38,63.0
3,佩戴,4,已回复,124.0,8.0,2022-08-01 10:04:48,37.5
4,佩戴,5,已回复,32.0,26.0,2022-07-31 22:29:27,26.0
...,...,...,...,...,...,...,...
71,酷动,14903,已回复,130.0,7.0,2022-07-31 21:19:57,24.0
72,酷动,14904,已回复,184.0,4.0,2022-07-31 18:22:59,17.0
73,酷动,14905,已回复,203.0,5.0,2022-07-31 17:49:33,20.0
74,酷动,14906,已回复,310.0,99.0,2022-07-31 15:44:26,54.5
